In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud,STOPWORDS
import missingno as msno
import squarify

%matplotlib inline
plt.style.use('fivethirtyeight')
plt.rcParams.update({'font.size':12})

In [3]:
#path = 'dataset/'
path = 'wine-reviews/'
wine = pd.read_csv(path + 'winemag-data-130k-v2.csv',index_col=0)
# wine_150k = pd.read_csv(path + 'winemag-data_first150k.csv',index_col=0)
# wine_130k = pd.read_csv(path + 'winemag-data-130k-v2.csv',index_col=0)
# wine = pd.concat([wine_150k,wine_130k],axis=0)
#wine = wine_150k
print("Number of rows and columns:",wine.shape)

Number of rows and columns: (129971, 13)


In [4]:
wine.head()

,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks


In [35]:
wine.isna().sum()# 280901

# array(['Kerin O’Keefe', 'Roger Voss', 'Paul Gregutt',
#        'Alexander Peartree', 'Michael Schachner', 'Anna Lee C. Iijima',
#        'Virginie Boone', 'Matt Kettmann', nan, 'Sean P. Sullivan',
#        'Jim Gordon', 'Joe Czerwinski', 'Anne Krebiehl\xa0MW',
#        'Lauren Buzzeo', 'Mike DeSimone', 'Jeff Jenssen',
#        'Susan Kostrzewa', 'Carrie Dykes', 'Fiona Adams',
#        'Christina Pickard'], dtype=object)

# Nicosia 2013 Vulkà Bianco  (Etna)
wine['title'].unique().shape
# wine.shape
wine.loc[wine['title'] == 'Nicosia 2013 Vulkà Bianco  (Etna)']# wine['tile'].

# wine['taster_name'] == 'Roger Voss'
# wine.loc[wine['taster_name'] == 'Roger Voss']

,country,description,designation,points,price,province,region_1,region_2,taster_name,title,variety,winery
0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,30.0,Sicily & Sardinia,Etna,Unknown,Kerin O’Keefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia


In [17]:
# drop columns
wine.drop(columns=['taster_twitter_handle'], inplace=True)
# drop duplicates
wine.drop_duplicates(inplace=True)
# add "Unknown" to missing values in the categories with text
wine.country.fillna("Unknown",inplace=True)
wine.designation.fillna("Unknown",inplace=True)
wine.province.fillna("Unknown",inplace=True)
wine.region_1.fillna("Unknown",inplace=True)
wine.region_2.fillna("Unknown",inplace=True)
# wine.taster_name.fillna("Unknown",inplace=True)
wine.title.fillna("Unknown",inplace=True)
# there is only one row with missing variety
wine.drop(wine[wine.variety.isna()].index, inplace=True)
# for rating matrix, kick out wine entries that dont have a tastername
wine.drop(wine[wine.taster_name.isna()].index, inplace=True)


wine.isna().sum() #86909

country           0
description       0
designation       0
points            0
price          6751
province          0
region_1          0
region_2          0
taster_name       0
title             0
variety           0
winery            0
dtype: int64

In [18]:
medians = wine.groupby(['country']).price.median()
medians

country
Argentina                 17.0
Armenia                   14.5
Australia                 22.0
Austria                   25.0
Bosnia and Herzegovina    12.5
Brazil                    22.0
Bulgaria                  14.0
Canada                    30.0
Chile                     15.0
China                     18.0
Croatia                   20.0
Cyprus                    16.5
Czech Republic            18.0
Egypt                      NaN
England                   50.0
France                    25.0
Georgia                   17.5
Germany                   27.0
Greece                    19.0
Hungary                   25.0
India                     12.0
Israel                    25.0
Italy                     30.0
Lebanon                   25.0
Luxembourg                23.0
Macedonia                 15.0
Mexico                    23.0
Moldova                   13.0
Morocco                   18.0
New Zealand               21.0
Peru                      14.5
Portugal                  17.0


In [19]:
# TODO BECAREFUL WITH "UNKNOWN", what to do with rows with no countries?

# Get the median price per country and fill n/a price values based on their country
wine['price'] = wine['price'].fillna(wine.groupby('country')['price'].transform('median'))

# add total median price for Egypt and Tunisia since they have no value prices information for their countries
wine['price'] = wine['price'].fillna(wine['price'].median())
print(wine.isna().sum())

country        0
description    0
designation    0
points         0
price          0
province       0
region_1       0
region_2       0
taster_name    0
title          0
variety        0
winery         0
dtype: int64


In [24]:
# wine.to_csv('./wine-reviews/winemag-data-130k-v4.csv', mode='w',header=True)
wine.to_csv('./wine-reviews/cleaned-data-with-tasters.csv', mode='w',header=True)

In [25]:
# wine = pd.read_csv('winemag-data-130k-v4.csv',index_col=0)
wine = pd.read_csv('./wine-reviews/cleaned-data-with-tasters.csv',index_col=0)
print("Number of rows and columns:",wine.shape)
wine.isna().sum()

Number of rows and columns: (95071, 12)


country        0
description    0
designation    0
points         0
price          0
province       0
region_1       0
region_2       0
taster_name    0
title          0
variety        0
winery         0
dtype: int64